In [287]:
import nltk
from nltk import word_tokenize
import pandas as pd
from itertools import chain 
import re
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
import joblib

In [288]:
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0]+','+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>,'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+','+content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [289]:
def read_dataset(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        print('idx_line =')
        print(idx_line)
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sent.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

In [956]:
list_test_sentences, list_test_tags = read_dataset('Testing.txt') 

idx_line =
0
idx_line =
11
idx_line =
20
idx_line =
28
idx_line =
40
idx_line =
50
idx_line =
68
idx_line =
78
idx_line =
87
idx_line =
101


## Metode Baseline

### Mencari Tag

Disini melakukan pencarian tag pada kata di satu kalimat, dengan menggunakan sebarapa banyak kata tersebut digunakan dengan tag tertentu. Tag yang terpilih adalah tag dengan frekuensi terbanyak dari kata tersebut.

In [400]:
def findTags(sentences, word_tag):
    tags = []
    for word in sentences:
        possible = []
        key = []
        possible = [(value, key) for key, value in word_tag.items() if word in key]
        for i in range(len(possible)):
            count, word_poss = possible[i]
            key_poss = word_poss.split(',')
            if len(key_poss)!=2:
                tmp = ''
                for j in range (len(key_poss)-1):
                    if j == len(key_poss)-2:
                        tmp += key_poss[j]
                    else:
                        tmp += key_poss[j] + ','
                for k in range((len(key_poss)-1)):
                    key_poss.remove(key_poss[0])
                key_poss.insert(0, tmp)
            if key_poss[0] == word:
                key.append(possible[i])
        key.sort(reverse=True)
        tag_poss = key[0][1].split(',')
        if len(tag_poss)==2:
            tags.append(tag_poss[1])
        else:
            tags.append(tag_poss[len(tag_poss)-1])

    return tags

In [977]:
def accuracy(predict_tag, actual_tag):
    count_a = 0
    count_d = 0
    for i in range(len(actual_tag)):
        for j in range(len(actual_tag[i])):
            if predict_tag[i][j] == actual_tag[i][j]:
                       count_a += 1
            count_d +=1

    return count_a/count_d

### Pengujian

Melakukan pengujian ke 10 kalimat yang sudah ditentukan

In [982]:
tag_count, word_tag, tag_trans = read_file_init_table('Indonesian_Manually_Tagged_Corpus_ID.tsv')
tag = []
for i in range (len(list_test_sentences)):
    sentences = list_test_sentences[i]
    tag.append(findTags(sentences, word_tag))
    print(i+1, sentences,'-',tag[i])
print()
print('Accuracy :', accuracy(tag, list_test_tags)*100,'%')

1 ['Saya', 'belum', 'tahu', 'siapa', 'yang', 'akan', 'menggantikan', 'saya'] - ['PRP', 'NEG', 'VB', 'WH', 'SC', 'MD', 'VB', 'PRP']
2 ['Monyet', 'besar', 'akan', 'dikerahkan', 'di', 'dua', 'stadion'] - ['NN', 'JJ', 'MD', 'VB', 'IN', 'CD', 'NN']
3 ['Roekman', 'mengatakan', 'pesawat', 'tersebut', 'akan', 'disewakan'] - ['NNP', 'VB', 'NN', 'PR', 'MD', 'VB']
4 ['pemerintah', 'juga', 'mencatat', 'total', 'belanja', 'kementerian', 'dan', 'lembaga pemerintah', 'setiap', 'tahun'] - ['NN', 'RB', 'VB', 'NN', 'NN', 'NNP', 'CC', 'NN', 'CD', 'NN']
5 ['Walaupun', 'kemarin', 'nilai tukar', 'mata', 'uang', 'Thailand', 'sempat', 'menurun'] - ['SC', 'NN', 'NN', 'NN', 'NN', 'NNP', 'MD', 'VB']
6 ['Sri', 'Mulyani', 'mengakui', 'setidaknya', 'ada', 'tiga', 'faktor', 'yang', 'memberi', 'risiko', 'terhadap', 'proyeksi', 'ekonomi', '2007', 'dan', '2008'] - ['NNP', 'NNP', 'VB', 'RB', 'VB', 'CD', 'NN', 'SC', 'VB', 'NN', 'IN', 'NN', 'NN', 'CD', 'CC', 'CD']
7 ['Realisasi', 'terhadap', 'rencana', 'tersebut', 'akan',

## Metode Statistika

In [3]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }

def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y

def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return tags

In [4]:
sentences,tags = read_dataset('Indonesian_Manually_Tagged_Corpus_ID.tsv')

idx_line =
0
idx_line =
7
idx_line =
27
idx_line =
46
idx_line =
63
idx_line =
86
idx_line =
105
idx_line =
127
idx_line =
145
idx_line =
159
idx_line =
189
idx_line =
198
idx_line =
213
idx_line =
227
idx_line =
241
idx_line =
257
idx_line =
274
idx_line =
289
idx_line =
296
idx_line =
319
idx_line =
337
idx_line =
354
idx_line =
377
idx_line =
392
idx_line =
419
idx_line =
439
idx_line =
455
idx_line =
476
idx_line =
499
idx_line =
516
idx_line =
539
idx_line =
546
idx_line =
560
idx_line =
585
idx_line =
601
idx_line =
622
idx_line =
655
idx_line =
689
idx_line =
706
idx_line =
738
idx_line =
765
idx_line =
776
idx_line =
792
idx_line =
828
idx_line =
834
idx_line =
856
idx_line =
875
idx_line =
899
idx_line =
919
idx_line =
960
idx_line =
984
idx_line =
1001
idx_line =
1017
idx_line =
1034
idx_line =
1053
idx_line =
1090
idx_line =
1111
idx_line =
1118
idx_line =
1151
idx_line =
1169
idx_line =
1186
idx_line =
1203
idx_line =
1222
idx_line =
1236
idx_line =
1275
idx_line =
1298
idx

idx_line =
37366
idx_line =
37382
idx_line =
37407
idx_line =
37426
idx_line =
37445
idx_line =
37464
idx_line =
37489
idx_line =
37499
idx_line =
37543
idx_line =
37578
idx_line =
37607
idx_line =
37651
idx_line =
37681
idx_line =
37709
idx_line =
37768
idx_line =
37797
idx_line =
37814
idx_line =
37835
idx_line =
37854
idx_line =
37880
idx_line =
37899
idx_line =
37942
idx_line =
37967
idx_line =
38006
idx_line =
38026
idx_line =
38049
idx_line =
38093
idx_line =
38157
idx_line =
38182
idx_line =
38219
idx_line =
38255
idx_line =
38285
idx_line =
38300
idx_line =
38341
idx_line =
38369
idx_line =
38403
idx_line =
38438
idx_line =
38472
idx_line =
38517
idx_line =
38548
idx_line =
38589
idx_line =
38637
idx_line =
38648
idx_line =
38697
idx_line =
38741
idx_line =
38777
idx_line =
38825
idx_line =
38872
idx_line =
38901
idx_line =
38916
idx_line =
38954
idx_line =
38976
idx_line =
39008
idx_line =
39043
idx_line =
39086
idx_line =
39111
idx_line =
39122
idx_line =
39155
idx_line =
391

idx_line =
73324
idx_line =
73353
idx_line =
73382
idx_line =
73425
idx_line =
73457
idx_line =
73482
idx_line =
73508
idx_line =
73525
idx_line =
73550
idx_line =
73566
idx_line =
73591
idx_line =
73619
idx_line =
73656
idx_line =
73689
idx_line =
73727
idx_line =
73751
idx_line =
73774
idx_line =
73815
idx_line =
73843
idx_line =
73864
idx_line =
73883
idx_line =
73910
idx_line =
73964
idx_line =
73981
idx_line =
74016
idx_line =
74092
idx_line =
74111
idx_line =
74129
idx_line =
74159
idx_line =
74187
idx_line =
74236
idx_line =
74284
idx_line =
74322
idx_line =
74348
idx_line =
74375
idx_line =
74391
idx_line =
74421
idx_line =
74449
idx_line =
74476
idx_line =
74516
idx_line =
74556
idx_line =
74603
idx_line =
74630
idx_line =
74654
idx_line =
74670
idx_line =
74693
idx_line =
74712
idx_line =
74751
idx_line =
74806
idx_line =
74833
idx_line =
74869
idx_line =
74897
idx_line =
74929
idx_line =
74954
idx_line =
74982
idx_line =
75013
idx_line =
75051
idx_line =
75074
idx_line =
751

108547
idx_line =
108573
idx_line =
108619
idx_line =
108644
idx_line =
108675
idx_line =
108709
idx_line =
108752
idx_line =
108795
idx_line =
108838
idx_line =
108871
idx_line =
108905
idx_line =
108941
idx_line =
108974
idx_line =
108999
idx_line =
109027
idx_line =
109045
idx_line =
109076
idx_line =
109104
idx_line =
109132
idx_line =
109173
idx_line =
109200
idx_line =
109231
idx_line =
109261
idx_line =
109290
idx_line =
109315
idx_line =
109353
idx_line =
109381
idx_line =
109408
idx_line =
109433
idx_line =
109474
idx_line =
109515
idx_line =
109533
idx_line =
109571
idx_line =
109604
idx_line =
109634
idx_line =
109654
idx_line =
109679
idx_line =
109700
idx_line =
109729
idx_line =
109763
idx_line =
109806
idx_line =
109838
idx_line =
109871
idx_line =
109890
idx_line =
109909
idx_line =
109939
idx_line =
109971
idx_line =
110004
idx_line =
110041
idx_line =
110073
idx_line =
110104
idx_line =
110139
idx_line =
110175
idx_line =
110222
idx_line =
110248
idx_line =
110276
idx

idx_line =
144299
idx_line =
144320
idx_line =
144349
idx_line =
144390
idx_line =
144414
idx_line =
144432
idx_line =
144453
idx_line =
144507
idx_line =
144538
idx_line =
144559
idx_line =
144591
idx_line =
144622
idx_line =
144663
idx_line =
144679
idx_line =
144699
idx_line =
144723
idx_line =
144743
idx_line =
144776
idx_line =
144792
idx_line =
144809
idx_line =
144835
idx_line =
144855
idx_line =
144877
idx_line =
144905
idx_line =
144942
idx_line =
144982
idx_line =
145016
idx_line =
145039
idx_line =
145068
idx_line =
145093
idx_line =
145117
idx_line =
145152
idx_line =
145177
idx_line =
145203
idx_line =
145231
idx_line =
145253
idx_line =
145273
idx_line =
145297
idx_line =
145310
idx_line =
145322
idx_line =
145339
idx_line =
145364
idx_line =
145389
idx_line =
145422
idx_line =
145448
idx_line =
145473
idx_line =
145491
idx_line =
145524
idx_line =
145539
idx_line =
145564
idx_line =
145600
idx_line =
145624
idx_line =
145640
idx_line =
145666
idx_line =
145690
idx_line =

177266
idx_line =
177286
idx_line =
177312
idx_line =
177342
idx_line =
177356
idx_line =
177366
idx_line =
177382
idx_line =
177401
idx_line =
177422
idx_line =
177459
idx_line =
177478
idx_line =
177502
idx_line =
177534
idx_line =
177556
idx_line =
177577
idx_line =
177607
idx_line =
177622
idx_line =
177652
idx_line =
177676
idx_line =
177690
idx_line =
177720
idx_line =
177747
idx_line =
177774
idx_line =
177795
idx_line =
177813
idx_line =
177825
idx_line =
177845
idx_line =
177867
idx_line =
177882
idx_line =
177909
idx_line =
177932
idx_line =
177963
idx_line =
177983
idx_line =
178001
idx_line =
178017
idx_line =
178044
idx_line =
178069
idx_line =
178082
idx_line =
178113
idx_line =
178147
idx_line =
178167
idx_line =
178190
idx_line =
178214
idx_line =
178229
idx_line =
178256
idx_line =
178283
idx_line =
178298
idx_line =
178317
idx_line =
178343
idx_line =
178357
idx_line =
178376
idx_line =
178389
idx_line =
178402
idx_line =
178424
idx_line =
178448
idx_line =
178472
idx

idx_line =
209702
idx_line =
209730
idx_line =
209770
idx_line =
209808
idx_line =
209837
idx_line =
209882
idx_line =
209899
idx_line =
209937
idx_line =
209970
idx_line =
210003
idx_line =
210045
idx_line =
210080
idx_line =
210111
idx_line =
210137
idx_line =
210167
idx_line =
210206
idx_line =
210224
idx_line =
210259
idx_line =
210295
idx_line =
210332
idx_line =
210366
idx_line =
210396
idx_line =
210409
idx_line =
210421
idx_line =
210444
idx_line =
210470
idx_line =
210510
idx_line =
210527
idx_line =
210545
idx_line =
210565
idx_line =
210582
idx_line =
210613
idx_line =
210632
idx_line =
210658
idx_line =
210684
idx_line =
210706
idx_line =
210733
idx_line =
210749
idx_line =
210760
idx_line =
210799
idx_line =
210820
idx_line =
210846
idx_line =
210881
idx_line =
210894
idx_line =
210908
idx_line =
210924
idx_line =
210951
idx_line =
210962
idx_line =
210997
idx_line =
211026
idx_line =
211052
idx_line =
211077
idx_line =
211099
idx_line =
211112
idx_line =
211129
idx_line =

idx_line =
243490
idx_line =
243520
idx_line =
243556
idx_line =
243583
idx_line =
243615
idx_line =
243642
idx_line =
243679
idx_line =
243706
idx_line =
243719
idx_line =
243742
idx_line =
243757
idx_line =
243795
idx_line =
243819
idx_line =
243850
idx_line =
243886
idx_line =
243921
idx_line =
243948
idx_line =
243965
idx_line =
243987
idx_line =
244011
idx_line =
244043
idx_line =
244075
idx_line =
244095
idx_line =
244109
idx_line =
244138
idx_line =
244156
idx_line =
244185
idx_line =
244199
idx_line =
244230
idx_line =
244264
idx_line =
244304
idx_line =
244340
idx_line =
244369
idx_line =
244387
idx_line =
244413
idx_line =
244456
idx_line =
244504
idx_line =
244512
idx_line =
244520
idx_line =
244547
idx_line =
244565
idx_line =
244597
idx_line =
244614
idx_line =
244650
idx_line =
244681
idx_line =
244712
idx_line =
244723
idx_line =
244752
idx_line =
244785
idx_line =
244815
idx_line =
244846
idx_line =
244894
idx_line =
244927
idx_line =
244957
idx_line =
244993
idx_line =

### Split Data

Melakukan split data menjadi data train(75%) dan data test(25%), dan juga pada data train dibuat menjadi 4 batch untuk menghindari out of memory (RAM dengan 16 GB bisa)

In [5]:
# Split the dataset for training and testing
cutoff = int(.75 * len(sentences))
training_sentences = sentences[:cutoff]
test_sentences = sentences[cutoff:]
training_tags = tags[:cutoff]
test_tags = tags[cutoff:]

# Split data train to 4 batch
X, y = transform_to_dataset(training_sentences, training_tags)
cut1, cut2, cut3 = int(.25 * len(X)), int(.5 * len(X)), int(.75 * len(X))
X1, X2, X3, X4 = X[:cut1], X[cut1:cut2], X[cut2:cut3], X[cut3:]
y1, y2, y3, y4 = y[:cut1], y[cut1:cut2], y[cut2:cut3], y[cut3:]

print('data training ke-0 =')
print(X1[0])
print('label training ke-0 =')
print(y[0])

data training ke-0 =
{'word': 'Kera', 'prefix-1': 'K', 'prefix-2': 'Ke', 'prefix-3': 'Ker', 'suffix-1': 'a', 'suffix-2': 'ra', 'suffix-3': 'era', 'prev_word': '', 'next_word': 'untuk'}
label training ke-0 =
NN


### Training


Pada bagian ini melakukan training, tetapi jika tidak ingin melakukannya (dikarenakan lama) saya sudah menyediakan load_model dibawah sehingga tidak harus melakukan training

In [6]:
#Melakukan Training
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X1, y1)
clf.fit(X2, y2)
clf.fit(X3, y3)
clf.fit(X4, y4)
 
print('Training completed')

Training completed


### Save Model

Agar jika ingin menggunakan model, tidak harus melakukan training lagi

In [19]:
# save the model to disk
filename = 'finalized_modelDecisionTree.sav'
joblib.dump(clf, filename)

['finalized_modelDecisionTree.sav']

### Testing data

Melakukan testing dengan data corpus yang sudah di split sebelumnya.

In [262]:
X_test, y_test = transform_to_dataset(test_sentences, test_tags) 

In [21]:
# Jika langsung dari training, Akurasi dari datatest
print("Accuracy dari pembagian dataset:"clf.score(X_test, y_test))

Accuracy:
0.9573373964384344


### Load Model

Dibagian ini melakukan load_model, lalu melakukan testing terhadap data test yang di split dari corpus

In [195]:
X_test, y_test = transform_to_dataset(test_sentences, test_tags) 
# load the model from disk, jika mengunakan save model
loaded_model = joblib.load('finalized_modelDecisionTree.sav')
result = loaded_model.score(X_test, y_test)
print(result)

0.9573373964384344


### Pengujian dengan 10 kalimat

In [972]:
sen_tags = []
for i in range (len(list_test_sentences)):
    sep = ' '
    sentences = sep.join(list_test_sentences[i])
    sen_tags.append(pos_tag(word_tokenize(sentences)))
    print(i+1, sentences,'-',sen_tags[i])

1 Saya belum tahu siapa yang akan menggantikan saya - ['PRP' 'NEG' 'VB' 'WH' 'SC' 'MD' 'VB' 'PRP']
2 Monyet besar akan dikerahkan di dua stadion - ['NN' 'JJ' 'MD' 'VB' 'IN' 'CD' 'NN']
3 Roekman mengatakan pesawat tersebut akan disewakan - ['NN' 'VB' 'NN' 'PR' 'MD' 'VB']
4 pemerintah juga mencatat total belanja kementerian dan lembaga pemerintah setiap tahun - ['NN' 'RB' 'VB' 'NN' 'NN' 'NN' 'CC' 'NN' 'NN' 'CD' 'NN']
5 Walaupun kemarin nilai tukar mata uang Thailand sempat menurun - ['SC' 'NN' 'NN' 'VB' 'NN' 'NN' 'NNP' 'MD' 'VB']
6 Sri Mulyani mengakui setidaknya ada tiga faktor yang memberi risiko terhadap proyeksi ekonomi 2007 dan 2008 - ['NN' 'NNP' 'VB' 'RB' 'VB' 'CD' 'NN' 'SC' 'VB' 'NN' 'IN' 'NN' 'NN' 'CD'
 'CC' 'CD']
7 Realisasi terhadap rencana tersebut akan dilakukan secara bertahap - ['NN' 'IN' 'NN' 'PR' 'MD' 'VB' 'IN' 'VB']
8 Sehingga yang dilakukan adalah memblokir kegiatan kepabeanan - ['SC' 'SC' 'VB' 'VB' 'VB' 'NN' 'NN']
9 Peningkatan laba ini ditunjang oleh adanya perkiraan 

In [983]:
X_testing, y_testing = transform_to_dataset(list_test_sentences, list_test_tags)
print('Accuracy dari 10 kalimat: ',clf.score(X_testing, y_testing)*100,'%')

Accuracy dari 10 kalimat:  96.62921348314607 %


## Metode HMM-Viterbi

Membuat trans_probabilities untuk semua POSTag yang ada

In [439]:
def create_trans_prob_table(tag_trans, tag_count):
#     print(tag_trans)
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            #print('tag1 = ')
            #print(tag1)
            trans_idx = tag1+','+tag2
            #print('trans_idx = ')
            #print(trans_idx)
            if trans_idx in tag_trans:
                #print(trans_idx)
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

Membuat emission_probabilities untuk semua kata dan POSTag yang ada

In [986]:
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split(',')
        if len(word_tag_split) != 2:
            tmp1 = word_tag_split
            tmp = ''
            for i in range(len(word_tag_split)-1):
                if i == len(word_tag_split):
                    tmp += tmp1[i]
                else:
                    tmp += tmp1[i]+','
            for i in range((len(word_tag_split)-1)):
                word_tag_split.remove(word_tag_split[0])
            word_tag_split.insert(0, tmp)
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        emission_key = current_word+','+current_tag
        emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

mendapat word dan tag yang dibutuhkan dari kalimat uji

In [765]:
def getWordTag(emission_prob, sentence_words):
    wordtag = []
    for word in sentence_words:
            key = []
            possible = []
            possible = [(key, value) for key, value in emission_prob.items() if word in key]
            for i in range(len(possible)):
                word_poss, val = possible[i]
                key_poss = word_poss.split(',')
                if len(key_poss)!=2:
                    tmp = ''
                    for j in range (len(key_poss)-1):
                        if j == len(key_poss)-2:
                            tmp += key_poss[j]
                        else:
                            tmp += key_poss[j] + ','
                    for k in range((len(key_poss)-1)):
                        key_poss.remove(key_poss[0])
                    key_poss.insert(0, tmp)
                if key_poss[0] == word:
                    key.append(possible[i])
            key.sort
            wordtag.append(key)
    return wordtag

Disini dapat dilakukan pengambilan tag nya saja dari wordtag, dan melakukan kombinasi kemungkinan tag yang digunakan dalam kalkulasi perhitungan

In [731]:
def takeTag(wordtag):
    tags = []
    tagsfus = []
    for i in range(len(wordtag)):
        tagsr = []
        for  j in range(len(wordtag[i])):
            a,b = wordtag[i][j]
            aa = a.split(',')
            tagsr.append(aa[1])
        tags.append(tagsr)

    return tags

def combineTags(tags):
    tmptg = []
    for i in range(len(tags)):
        tmp = []
        for j in range(len(tags[i])):
            if i==0:
                tmp.append('<start>'+','+tags[i][j])
            else:
                for k in range(len(tm)):
                    tmp.append(tm[k]+','+tags[i][j])
        
        tmptg.append(tmp)
        tm = tags[i]
        
    return tmptg

Melakukan Kalkulasi perhitungan viberti

In [912]:
def calculation(wordtag, taglist):
    cek = []
    for i in range(len(wordtag)):
        group = []
        for j in range(len(wordtag[i])):
            if i == 0:
                group.append(wordtag[i][0][1]*taglist[i][j][1])
            else:
                count = [0 * x for x in range(len(wordtag[i]))]
                length = len(taglist[i])//len(wordtag[i])
                if length==1:
                    for k in range(length):
                        count[0] += cekbfr[k]*wordtag[i][j][1]*taglist[i][k][1]
                else:
                    for k in range(len(count)):
                        for cnt in range(length):
                            if k % 5 == 0:
                                count[0] += cekbfr[k]*wordtag[i][j][1]*taglist[i][cnt][1]
                            elif k % 5 == 1:
                                count[1] += cekbfr[k]*wordtag[i][j][1]*taglist[i][cnt][1]
                            elif k % 5 == 2:
                                count[2] += cekbfr[k]*wordtag[i][j][1]*taglist[i][cnt][1]
                            elif k % 5 == 3:
                                count[3] += cekbfr[k]*wordtag[i][j][1]*taglist[i][cnt][1]
                            elif k % 5 == 4:
                                count[4] += cekbfr[k]*wordtag[i][j][1]*taglist[i][cnt][1]
                res = 0
                for k in range (len(count)):
                    res += count[k]
                group.append(res)      
        cekbfr = group
        cek.append(group)
    
    return cek

Prosedur viterbinya

In [917]:
def viterbi(trans_prob, emission_prob, tag_count, sentence):
    tag_sequence = []
    
    sentence_words = sentence.split()
    wordtag = getWordTag(emission_prob, sentence_words)
    tagRelation = combineTags(takeTag(wordtag))
    
    taglist = []
    for i in range(len(tagRelation)):
        tm = []
        for j in range(len(tagRelation[i])):
            possible = [(key, value) for key, value in trans_prob.items() if tagRelation[i][j] == key]
            if possible == []:
                tm.append([(tagRelation[i][j], 0.0)])
            else:
                tm.append(possible[0])

        taglist.append(tm)
        
    getCal = calculation(wordtag, taglist)
    choosen = []

    for i in range(len(wordtag)):
        choosen.append([wordtag[i][getCal[i].index(max(getCal[i]))]])
    
    choosen = takeTag(choosen)
    
    for i in range(len(choosen)):
        tag_sequence.append(choosen[i][0])
    
        
    return tag_sequence

In [60]:
tag_count, word_tag, tag_trans = read_file_init_table('Indonesian_Manually_Tagged_Corpus_ID.tsv')
print(tag_count)
print(word_tag)
print(tag_trans)

{'<start>': 10030, 'NN': 61940, 'SC': 13080, 'VB': 31733, 'NNP': 34649, 'JJ': 9724, 'RB': 4903, 'IN': 21311, 'Z': 26347, 'CD': 17819, 'CC': 7438, 'PR': 5348, 'PRP': 7583, 'MD': 5248, 'FW': 2365, 'NEG': 1520, 'DT': 381, 'NND': 1414, 'SYM': 2210, 'RP': 183, 'WH': 260, 'OD': 738, 'X': 397, 'UH': 30, 'fw': 1}
{'Kera,NN': 1, 'untuk,SC': 1706, 'amankan,VB': 3, 'pesta olahraga,NN': 1, 'Pemerintah,NNP': 60, 'kota,NNP': 46, 'Delhi,NNP': 12, 'mengerahkan,VB': 3, 'monyet,NN': 12, 'mengusir,VB': 11, 'monyet-monyet,NN': 2, 'lain,JJ': 453, 'yang,SC': 5616, 'berbadan,VB': 4, 'lebih,RB': 701, 'kecil,JJ': 134, 'dari,IN': 2345, 'arena,NN': 2, 'Pesta Olahraga,NNP': 1, 'Persemakmuran,NNP': 1, '.,Z': 9921, 'Beberapa,CD': 47, 'laporan,NN': 163, 'menyebutkan,VB': 91, 'setidaknya,RB': 11, '10,CD': 171, 'ditempatkan,VB': 24, 'di,IN': 4078, 'luar,NN': 62, 'lomba,NN': 1, 'dan,CC': 4565, 'pertandingan,NN': 1, 'ibukota,NNP': 2, 'India,NNP': 110, 'Pemkot,NNP': 1, 'memiliki,VB': 265, '28,CD': 31, 'berencana,VB': 53,

In [440]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)
print(trans_prob)

{'<start>,NN': 0.32083748753738783, '<start>,SC': 0.08195413758723828, '<start>,VB': 0.03529411764705882, '<start>,NNP': 0.17357926221335993, '<start>,JJ': 0.006879361914257229, '<start>,RB': 0.01585244267198405, '<start>,IN': 0.13110667996011965, '<start>,Z': 0.003389830508474576, '<start>,CD': 0.04187437686939183, '<start>,CC': 0.05732801595214357, '<start>,PR': 0.015653040877367897, '<start>,PRP': 0.07976071784646062, '<start>,MD': 0.003589232303090728, '<start>,FW': 0.0029910269192422734, '<start>,NEG': 0.0055832502492522435, '<start>,DT': 0.008773678963110668, '<start>,NND': 0.011166500498504487, '<start>,RP': 9.970089730807577e-05, '<start>,WH': 0.0019940179461615153, '<start>,OD': 0.0006979062811565304, '<start>,X': 0.0004985044865403788, '<start>,UH': 0.0010967098703888335, 'NN,NN': 0.2909428479173394, 'NN,SC': 0.07445915402001937, 'NN,VB': 0.057426541814659345, 'NN,NNP': 0.09899903132063287, 'NN,JJ': 0.07374878915079108, 'NN,RB': 0.015466580561834033, 'NN,IN': 0.08936067161769

In [445]:
emission_prob = create_emission_prob_table(word_tag, tag_count)
print(emission_prob)

{'Kera,NN': 1.614465611882467e-05, 'untuk,SC': 0.13042813455657493, 'amankan,VB': 9.45388081807582e-05, 'pesta olahraga,NN': 1.614465611882467e-05, 'Pemerintah,NNP': 0.0017316517071199746, 'kota,NNP': 0.001327599642125314, 'Delhi,NNP': 0.0003463303414239949, 'mengerahkan,VB': 9.45388081807582e-05, 'monyet,NN': 0.00019373587342589604, 'mengusir,VB': 0.0003466422966627801, 'monyet-monyet,NN': 3.228931223764934e-05, 'lain,JJ': 0.046585767174002465, 'yang,SC': 0.42935779816513764, 'berbadan,VB': 0.00012605174424101093, 'lebih,RB': 0.1429736895778095, 'kecil,JJ': 0.013780337309749074, 'dari,IN': 0.11003707005771667, 'arena,NN': 3.228931223764934e-05, 'Pesta Olahraga,NNP': 2.886086178533291e-05, 'Persemakmuran,NNP': 2.886086178533291e-05, '.,Z': 0.37655141002770715, 'Beberapa,CD': 0.0026376339861945114, 'laporan,NN': 0.002631578947368421, 'menyebutkan,VB': 0.0028676771814829987, 'setidaknya,RB': 0.0022435243728329596, '10,CD': 0.009596498119984287, 'ditempatkan,VB': 0.0007563104654460656, 'd

melakukan pengujian ke 10 kalimat yang telah ditentukan

In [980]:
result = []
for i in range(len(list_test_sentences)):
    sentence = sep.join(list_test_sentences[i])
    result.append(viterbi(trans_prob, emission_prob, tag_count, sentence))
    print(i+1,sentence,'-',result[i])

1 Saya belum tahu siapa yang akan menggantikan saya - ['PRP', 'NEG', 'VB', 'WH', 'SC', 'MD', 'VB', 'PRP']
2 Monyet besar akan dikerahkan di dua stadion - ['NN', 'JJ', 'MD', 'VB', 'IN', 'CD', 'NN']
3 Roekman mengatakan pesawat tersebut akan disewakan - ['NNP', 'VB', 'NN', 'PR', 'MD', 'VB']
4 pemerintah juga mencatat total belanja kementerian dan lembaga pemerintah setiap tahun - ['NN', 'RB', 'VB', 'NN', 'NN', 'NNP', 'CC', 'NN', 'NN', 'CD', 'NN']
5 Walaupun kemarin nilai tukar mata uang Thailand sempat menurun - ['SC', 'NN', 'NN', 'VB', 'NN', 'NN', 'NNP', 'MD', 'VB']
6 Sri Mulyani mengakui setidaknya ada tiga faktor yang memberi risiko terhadap proyeksi ekonomi 2007 dan 2008 - ['NNP', 'NNP', 'VB', 'RB', 'VB', 'CD', 'NN', 'SC', 'VB', 'NN', 'IN', 'NN', 'NN', 'CD', 'CC', 'CD']
7 Realisasi terhadap rencana tersebut akan dilakukan secara bertahap - ['NN', 'IN', 'NN', 'PR', 'MD', 'VB', 'IN', 'VB']
8 Sehingga yang dilakukan adalah memblokir kegiatan kepabeanan - ['SC', 'SC', 'VB', 'VB', 'VB', '

In [985]:
print('Accuracy Viterbi: ',accuracy(result, list_test_tags)*100,'%')

Accuracy Viterbi:  91.01123595505618 %
